In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch

# Đọc dữ liệu
train_df = pd.read_csv('/kaggle/input/vnews8td-main/VNews8td-main/train.tsv', sep='\t', names=['text', 'label'])
val_df = pd.read_csv('/kaggle/input/vnews8td-main/VNews8td-main/val.tsv', sep='\t', names=['text', 'label'])
test_df = pd.read_csv('/kaggle/input/vnews8td-main/VNews8td-main/test.tsv', sep='\t', names=['text', 'label'] )

# Kết hợp dữ liệu
df = pd.concat([train_df, val_df, test_df])


In [2]:
train_df.head()

,text,label
0,Ba Lan sa thải HLV Santos sau sáu trận. Liên đ...,thethao
1,Trữ trứng để sinh con được bao lâu?. Tôi 32 tu...,suckhoe
2,Thường xuyên đau lưng là bệnh gì?. Lưng vợ tôi...,suckhoe
3,Người mẹ bỏ chữa ung thư để tìm sự sống cho co...,doisong
4,"Nguy cơ Covid đồng nhiễm virus, vi khuẩn gây b...",suckhoe


Tiền xử lý DL


In [3]:
# Mã hóa nhãn
unique_labels = np.unique(df['label'])
num_class = len(unique_labels)
lEnc = LabelEncoder()
lEnc.fit(unique_labels)

print(unique_labels)
print(lEnc.transform(unique_labels))

train_labels = lEnc.transform(train_df['label'])
val_labels = lEnc.transform(val_df['label'])
test_labels = lEnc.transform(test_df['label'])

# Chuyển nhãn thành tensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels = train_labels.tolist() + val_labels.tolist() + test_labels.tolist()
labels = torch.LongTensor(labels).to(device)

['doisong' 'giaoduc' 'khoahoc' 'kinhte' 'suckhoe' 'thegioi' 'thethao'
 'thoisu']
[0 1 2 3 4 5 6 7]


In [4]:
from string import punctuation
import re

def vi_clean_str(input_str: str) -> str:
    my_punctuation = punctuation.replace('_', '')
    output_str = input_str.translate(str.maketrans('', '', my_punctuation))
    output_str = output_str.replace('_,', '')
    output_str = output_str.replace('_.', '')
    output_str = output_str.replace('…', '')
    output_str = output_str.replace('-', '_')
    output_str = output_str.replace('–', '_')
    output_str = output_str.replace('\u200b_\u200b', '')
    output_str = output_str.replace('\u200b', '')
    output_str = output_str.replace('‘', '_')
    output_str = output_str.replace('’', '_')
    output_str = output_str.replace('“', '_')
    output_str = output_str.replace('”', '_')
    output_str = ' '.join(output_str.split())
    return output_str.lower()

train_df['clean_text'] = train_df['text'].apply(vi_clean_str)
val_df['clean_text'] = val_df['text'].apply(vi_clean_str)
test_df['clean_text'] = test_df['text'].apply(vi_clean_str)


In [5]:
train_df['clean_text']

0        ba lan sa thải hlv santos sau sáu trận liên đo...
1        trữ trứng để sinh con được bao lâu tôi 32 tuổi...
2        thường xuyên đau lưng là bệnh gì lưng vợ tôi t...
3        người mẹ bỏ chữa ung thư để tìm sự sống cho co...
4        nguy cơ covid đồng nhiễm virus vi khuẩn gây bệ...
                               ...                        
31740    nhật bản ra sức tìm lại vàng son ngành chip ch...
31741    ông bùi hoàng phương làm thứ trưởng thông tin ...
31742    cựu vô địch major đòi tẩy chay giải golf náo n...
31743    14 triết lý sống của người nhật người nhật luô...
31744    ngậm cam chanh có giảm ngủ ngáy tôi nghe mọi n...
Name: clean_text, Length: 31745, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo từ điển từ các văn bản
all_texts = train_df['clean_text'].tolist() + val_df['clean_text'].tolist() + test_df['clean_text'].tolist()
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(all_texts)

# Tạo đặc trưng cho các tập
X_train = vectorizer.transform(train_df['clean_text']).toarray()
X_val = vectorizer.transform(val_df['clean_text']).toarray()
X_test = vectorizer.transform(test_df['clean_text']).toarray()

# Chuyển thành tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long).to(device)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long).to(device)


Huấn luyện mô hình

In [7]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Định nghĩa mô hình CNN
class CNN(nn.Module):
    def __init__(self, num_features, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=100, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3, padding=1)
        self.fc = nn.Linear(num_features * 100, num_classes)
        
    def forward(self, x):
        x = x.unsqueeze(1)  # Thêm kênh vào
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

num_features = X_train_tensor.shape[1]
num_classes = num_class
model = CNN(num_features, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Tạo DataLoader
train_dataset = TensorDataset(X_train_tensor, train_labels_tensor)
val_dataset = TensorDataset(X_val_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)




In [18]:
# Huấn luyện mô hình
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Đánh giá mô hình
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                pred = outputs.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()
        
        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)
        
        print(f'Epoch: {epoch + 1}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}')

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)


Epoch: 1, Val Loss: 0.0052, Val Accuracy: 0.8895
Epoch: 2, Val Loss: 0.0044, Val Accuracy: 0.9105
Epoch: 3, Val Loss: 0.0046, Val Accuracy: 0.9058
Epoch: 4, Val Loss: 0.0052, Val Accuracy: 0.8982
Epoch: 5, Val Loss: 0.0062, Val Accuracy: 0.8976
Epoch: 6, Val Loss: 0.0076, Val Accuracy: 0.8913
Epoch: 7, Val Loss: 0.0091, Val Accuracy: 0.8924
Epoch: 8, Val Loss: 0.0103, Val Accuracy: 0.8884
Epoch: 9, Val Loss: 0.0116, Val Accuracy: 0.8911
Epoch: 10, Val Loss: 0.0124, Val Accuracy: 0.8915
Epoch: 11, Val Loss: 0.0135, Val Accuracy: 0.8895
Epoch: 12, Val Loss: 0.0141, Val Accuracy: 0.8915
Epoch: 13, Val Loss: 0.0148, Val Accuracy: 0.8900
Epoch: 14, Val Loss: 0.0153, Val Accuracy: 0.8911
Epoch: 15, Val Loss: 0.0155, Val Accuracy: 0.8869
Epoch: 16, Val Loss: 0.0153, Val Accuracy: 0.8837
Epoch: 17, Val Loss: 0.0135, Val Accuracy: 0.8811
Epoch: 18, Val Loss: 0.0146, Val Accuracy: 0.8846
Epoch: 19, Val Loss: 0.0160, Val Accuracy: 0.8846
Epoch: 20, Val Loss: 0.0167, Val Accuracy: 0.8840


In [25]:
from sklearn.metrics import classification_report
def evaluate_model(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    return y_true, y_pred

y_true, y_pred = evaluate_model(model, test_loader)
print(classification_report(y_true, y_pred, target_names=lEnc.classes_))

              precision    recall  f1-score   support

     doisong       0.62      0.62      0.62       473
     giaoduc       0.84      0.85      0.85       637
     khoahoc       0.80      0.78      0.79       595
      kinhte       0.81      0.82      0.81       867
     suckhoe       0.94      0.93      0.93      1917
     thegioi       0.91      0.91      0.91      1966
     thethao       0.99      0.98      0.98      1626
      thoisu       0.80      0.83      0.81      1034

    accuracy                           0.88      9115
   macro avg       0.84      0.84      0.84      9115
weighted avg       0.88      0.88      0.88      9115



In [29]:
# Câu cần dự đoán
new_sentence = "Đà Nẵng xem xét dừng hoạt động khách sạn xả thải ra biển. Chủ tịch TP Đà Nẵng Lê Trung Chinh nói sẽ dừng hoạt động nhà hàng, khách sạn không chịu đấu nối hệ thống xử lý nước thải, xả chui ra môi trường biển."
preprocessed_sentence = preprocess_text(new_sentence)
# Chuyển câu mới thành vector đặc trưng
new_sentence_vector = vectorizer.transform([preprocessed_sentence]).toarray()
new_sentence_tensor = torch.tensor(new_sentence_vector, dtype=torch.float).to(device)
# Dự đoán nhãn của câu mới
model.eval()
with torch.no_grad():
    output = model(new_sentence_tensor)
    predicted_label = output.argmax(dim=1).cpu().numpy()[0]

# Chuyển nhãn dự đoán từ số thành tên nhãn
predicted_label_name = lEnc.inverse_transform([predicted_label])[0]
print(f'Dự đoán nhãn: {predicted_label_name}')



Dự đoán nhãn: thoisu
